In [1]:
"""
Depeg Sentinel — unified entrypoint (notebook-safe)
Modes:
  - api   : serve the FastAPI app via uvicorn
  - loop  : run the sampling/scoring/training loop
  - batch : single pass sample/score (and optional alert)
  - test  : quick smoke run that exercises imports & a single sample
Notebook fix:
  In Jupyter/IPython a running asyncio event loop exists, which makes
  `uvicorn.run()` raise "asyncio.run() cannot be called from a running
  event loop". This entrypoint detects that and starts Uvicorn in a
  **background thread** instead of blocking the main loop.
"""
from __future__ import annotations
import argparse
import importlib
import os
import sys
import time
import traceback
import threading
from types import ModuleType
from typing import Optional

In [2]:
def build_parser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(prog="depeg-entrypoint")
    p.add_argument(
        "--mode",
        choices={"api", "loop", "batch", "test"},
        default=os.getenv("ENTRYPOINT_MODE", "api"),
        help="Which runtime to start",
    )
    p.add_argument("--host", default=os.getenv("HOST", "0.0.0.0"))
    p.add_argument("--port", type=int, default=int(os.getenv("PORT", "8000")))
    p.add_argument("--log-level", default=os.getenv("LOG_LEVEL", "info"))
    p.add_argument(
        "--app-path",
        default=os.getenv("APP_PATH", ""),  
        help="Import path for FastAPI app object (module:var). If empty, best-effort discovery.",
    )
    p.add_argument("--iters", type=int, default=int(os.getenv("ITERS", "30")))
    p.add_argument("--sleep", type=float, default=float(os.getenv("SLEEP", "60")))
    p.add_argument(
        "--module",
        default=os.getenv("SENTINEL_MODULE", "sentinel"),
        help="Module exposing runtime fns (job_sample_and_score_fast, train_if_needed, trigger_alerts_if_needed).",
    )
    p.add_argument(
        "--webhook",
        default=os.getenv("ALERT_WEBHOOK", "").strip(),
        help="Optional alert webhook override (else uses env ALERT_WEBHOOK).",
    )
    p.add_argument("--mock", action="store_true", default=("true" == os.getenv("MOCK_MODE", "").lower()))
    return p

In [3]:
def _try_import(module_path: str) -> Optional[ModuleType]:
    try:
        return importlib.import_module(module_path)
    except Exception:
        return None
def _in_notebook() -> bool:
    try:
        from IPython import get_ipython  
        return get_ipython() is not None
    except Exception:
        return False
def _event_loop_running() -> bool:
    try:
        import asyncio
        asyncio.get_running_loop()
        return True
    except Exception:
        return False

In [4]:
def _discover_app(app_path: str):
    """
    Returns a FastAPI app object. Prefers --app-path=X:Y, then common fallbacks.
    If not found, returns a tiny app with /healthz.
    """
    try:
        from fastapi import FastAPI
    except Exception as e:
        raise RuntimeError("fastapi is not installed") from e
    if app_path:
        mod_name, _, var_name = app_path.partition(":")
        if var_name:
            mod = _try_import(mod_name)
            if mod and hasattr(mod, var_name):
                return getattr(mod, var_name)
    for cand in ("sentinel.mcp", "mcp", "app"):
        mod = _try_import(cand)
        if mod and hasattr(mod, "app"):
            return getattr(mod, "app")
    app = FastAPI(title="Depeg Sentinel (minimal)", version="0.0.0")
    @app.get("/healthz")
    def healthz():
        return {"ok": True}
    return app

In [5]:
def _load_runtime_module(name: str) -> ModuleType:
    mod = _try_import(name)
    if mod:
        return mod
    for cand in ("depeg." + name, "src." + name, "app." + name):
        mod = _try_import(cand)
        if mod:
            return mod
    raise ImportError(
        f"Could not import runtime module '{name}'. "
        "Pass --module <path> or set SENTINEL_MODULE env var."
    )
def _get_any(mod: ModuleType, names: list[str]):
    for n in names:
        fn = getattr(mod, n, None)
        if callable(fn):
            return fn
    return None

In [6]:
def run_api(host: str, port: int, log_level: str, app_path: str) -> None:
    app = _discover_app(app_path)
    try:
        import uvicorn
    except Exception as e:
        print("[entrypoint] uvicorn is not installed:", e, file=sys.stderr)
        sys.exit(1)
    if _in_notebook() or _event_loop_running():
        def _serve():
            uvicorn.run(app, host=host, port=port, log_level=log_level)
        t = threading.Thread(target=_serve, name="uvicorn-thread", daemon=True)
        t.start()
        print(f"[entrypoint] api serving on http://{host}:{port} (background thread; notebook-safe)")
        return
    uvicorn.run(app, host=host, port=port, log_level=log_level)

In [7]:
def run_loop(
    module_name: str,
    iters: int,
    sleep_s: float,
    webhook: str,
    mock: bool,
) -> None:
    mod = _load_runtime_module(module_name)
    step = _get_any(mod, ["job_sample_and_score_fast", "job_sample_and_score", "sample_once_parallel", "sample_once"])
    train = _get_any(mod, ["train_if_needed", "train_models", "train"])
    alerts = _get_any(mod, ["trigger_alerts_if_needed", "trigger_alerts"])
    set_mock = _get_any(mod, ["set_mock_mode", "enable_mock_mode"])
    if mock and set_mock:
        try:
            set_mock(True)
            print("[entrypoint] mock mode enabled")
        except Exception:
            pass
    if not step:
        raise RuntimeError("No sampling/step function found (job_sample_and_score_fast/job_sample_and_score/sample_once(_parallel)).")
    TRAIN_EVERY = 240  
    loop_i = 0
    print(f"[entrypoint] loop start — iters={iters}, sleep={sleep_s}s, module={module_name}")
    try:
        while iters <= 0 or loop_i < iters:
            loop_i += 1
            try:
                step()
            except Exception as e:
                print(f"[entrypoint] step error: {e}")
                traceback.print_exc()
            if train and (loop_i % TRAIN_EVERY == 0):
                try:
                    train(force=False, min_hours=4.0, min_new_rows=400, label_drift_thr=0.15)  
                except TypeError:
                    try:
                        train()  
                    except Exception as e:
                        print("[entrypoint] train error:", e)
            if webhook and alerts and (loop_i % 3 == 0):
                try:
                    alerts(webhook, n_tail=80, min_rows_for_incidents=60)  
                except TypeError:
                    try:
                        alerts(webhook)  
                    except Exception as e:
                        print("[entrypoint] alert error:", e)
            if sleep_s > 0:
                time.sleep(sleep_s)
    except KeyboardInterrupt:
        print("\n[entrypoint] loop interrupted by user")

In [8]:
def run_batch(module_name: str, webhook: str, mock: bool) -> None:
    mod = _load_runtime_module(module_name)
    step = _get_any(mod, ["job_sample_and_score_fast", "job_sample_and_score", "sample_once_parallel", "sample_once"])
    alerts = _get_any(mod, ["trigger_alerts_if_needed", "trigger_alerts"])
    set_mock = _get_any(mod, ["set_mock_mode", "enable_mock_mode"])
    if mock and set_mock:
        try:
            set_mock(True)
            print("[entrypoint] mock mode enabled")
        except Exception:
            pass
    if not step:
        raise RuntimeError("No sampling/step function found.")
    print("[entrypoint] batch: single pass")
    step()
    if webhook and alerts:
        try:
            alerts(webhook, n_tail=80, min_rows_for_incidents=60)  
        except TypeError:
            alerts(webhook)  

In [9]:
def run_test(module_name: str) -> None:
    mod = _load_runtime_module(module_name)
    step = _get_any(mod, ["job_sample_and_score_fast", "job_sample_and_score", "sample_once_parallel", "sample_once"])
    if not step:
        raise RuntimeError("Smoke test failed: no sampler found.")
    print("[entrypoint] test: running a single step…")
    step()
    print("[entrypoint] test: OK")

In [10]:
def run(mode: str, iters: int, sleep: float, host: str, port: int, log_level: str, app_path: str, module: str, webhook: str, mock: bool) -> None:
    if mode == "api":
        run_api(host=host, port=port, log_level=log_level, app_path=app_path)
    elif mode == "loop":
        run_loop(module_name=module, iters=iters, sleep_s=sleep, webhook=webhook, mock=mock)
    elif mode == "batch":
        run_batch(module_name=module, webhook=webhook, mock=mock)
    elif mode == "test":
        run_test(module_name=module)
    else:
        raise SystemExit(f"Unknown mode: {mode}")

In [11]:
def main(argv: Optional[list[str]] = None) -> None:
    parser = build_parser()
    if argv is None:
        args, unknown = parser.parse_known_args()
        if unknown:
            print("[entrypoint] ignoring unknown args:", unknown)
    else:
        args = parser.parse_args(argv)
    run(
        mode=args.mode,
        iters=args.iters,
        sleep=args.sleep,
        host=args.host,
        port=args.port,
        log_level=args.log_level,
        app_path=args.app_path,
        module=args.module,
        webhook=args.webhook,
        mock=args.mock,
    )
if __name__ == "__main__":
    main()

[entrypoint] ignoring unknown args: ['-f', 'C:\\Users\\aniru\\AppData\\Roaming\\jupyter\\runtime\\kernel-baab81c9-c20b-4ad9-b001-5aa104f2dbdd.json']
[entrypoint] api serving on http://0.0.0.0:8000 (background thread; notebook-safe)


INFO:     Started server process [19428]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
